# **ETL Reviews usuarios de Yelp**

In [2]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
from datetime import datetime
import statsmodels.api as sm

## **Extraccion de los datos(desde la fuente)**

In [3]:
df_y1=pd.read_parquet('/content/drive/MyDrive/P_F_H/user.parquet')
df_y1.head(2)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946


In [4]:
df_y1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105597 entries, 0 to 2105596
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   name                object 
 2   review_count        int64  
 3   yelping_since       object 
 4   useful              int64  
 5   funny               int64  
 6   cool                int64  
 7   elite               object 
 8   friends             object 
 9   fans                int64  
 10  average_stars       float64
 11  compliment_hot      int64  
 12  compliment_more     int64  
 13  compliment_profile  int64  
 14  compliment_cute     int64  
 15  compliment_list     int64  
 16  compliment_note     int64  
 17  compliment_plain    int64  
 18  compliment_cool     int64  
 19  compliment_funny    int64  
 20  compliment_writer   int64  
 21  compliment_photos   int64  
dtypes: float64(1), int64(16), object(5)
memory usage: 353.4+ MB


## **Tranformacion de los datos**

In [9]:
df_y1[(df_y1['review_count']>=10) & (df_y1['useful']>=10)]

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2105591,Ss62yaJCCrxLgBkkUhXAQw,Carla,96,2015-04-09 23:28:43,194,61,100,"2015,2016","K64T_QAzJtGPnYe-obHHyA, mDuub5PwGKfGkgOLqPWTmw...",6,...,0,0,0,0,4,22,2,2,2,1
2105592,4QGxxakRZeOlg_qDuxmTeQ,Jennilee,38,2012-01-19 23:33:02,74,9,6,,kmwNG5LZSHFmveg6wYYdrw,0,...,1,0,0,0,1,4,0,0,1,0
2105593,tmelBbVBGAzXBVfH2u_R6g,Gerry,19,2009-06-09 16:34:54,14,5,2,,"BFYdCAMFyjYHDwesndEXEg, _9fTIqfSJc7g3V_o76XRVg...",1,...,1,0,0,0,0,1,0,0,0,0
2105595,Kst_srPw7GdYydMFYdCtzw,Heatheranne,25,2015-01-10 00:06:25,21,2,5,,"dzHTk52vbGtbktRm_B-wEg, fOfFLV7IbBDN6lzARaLqdg...",0,...,0,0,0,0,0,1,0,0,0,0


In [10]:
df_y1.drop(['name','compliment_hot','compliment_more','compliment_profile','compliment_cute','compliment_list','compliment_note','compliment_plain','compliment_cool','compliment_funny','compliment_writer','compliment_photos'], axis=1,inplace=True)

In [11]:
df_y1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105597 entries, 0 to 2105596
Data columns (total 10 columns):
 #   Column         Dtype  
---  ------         -----  
 0   user_id        object 
 1   review_count   int64  
 2   yelping_since  object 
 3   useful         int64  
 4   funny          int64  
 5   cool           int64  
 6   elite          object 
 7   friends        object 
 8   fans           int64  
 9   average_stars  float64
dtypes: float64(1), int64(5), object(4)
memory usage: 160.6+ MB


In [12]:
df_y1.drop(['useful','funny','cool','fans','friends'], axis=1,inplace=True)

In [13]:
df_y1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105597 entries, 0 to 2105596
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   user_id        object 
 1   review_count   int64  
 2   yelping_since  object 
 3   elite          object 
 4   average_stars  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 80.3+ MB


### **Busqueda de valores nulos**

In [14]:
df_y1.isnull().sum()

user_id          0
review_count     0
yelping_since    0
elite            0
average_stars    0
dtype: int64

In [15]:
df_y1.head(5)

,user_id,review_count,yelping_since,elite,average_stars
0,qVc8ODYU5SZjKXVBgXdI7w,585,2007-01-25 16:47:26,2007,3.91
1,j14WgRoU_-2ZE1aw1dXrJg,4333,2009-01-25 04:35:42,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...",3.74
2,2WnXYQFK0hXEoTxPtV2zvg,665,2008-07-25 10:41:00,"2009,2010,2011,2012,2013",3.32
3,SZDeASXq7o05mMNLshsdIA,224,2005-11-29 04:38:33,"2009,2010,2011",4.27
4,hA5lMy-EnncsH4JoR-hFGQ,79,2007-01-05 19:40:59,,3.54


In [17]:
df_y1.loc[1].elite
#Posible error al cargar año 2020

'2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,20,20,2021'

### **Busqueda de filas (valores) duplicados**

Realizo un conteo las filas duplicadas

In [18]:
df_y1.duplicated().sum()

117700

Visualiso las filas duplicadas

In [19]:
df_y1[df_y1.duplicated()]

,user_id,review_count,yelping_since,elite,average_stars
1987897,qVc8ODYU5SZjKXVBgXdI7w,585,2007-01-25 16:47:26,2007,3.91
1987898,j14WgRoU_-2ZE1aw1dXrJg,4333,2009-01-25 04:35:42,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...",3.74
1987899,2WnXYQFK0hXEoTxPtV2zvg,665,2008-07-25 10:41:00,"2009,2010,2011,2012,2013",3.32
1987900,SZDeASXq7o05mMNLshsdIA,224,2005-11-29 04:38:33,"2009,2010,2011",4.27
1987901,hA5lMy-EnncsH4JoR-hFGQ,79,2007-01-05 19:40:59,,3.54
...,...,...,...,...,...
2105592,4QGxxakRZeOlg_qDuxmTeQ,38,2012-01-19 23:33:02,,2.98
2105593,tmelBbVBGAzXBVfH2u_R6g,19,2009-06-09 16:34:54,,3.68
2105594,tpBznnD6uJN3m_pJubj09w,26,2013-08-13 23:18:11,,3.85
2105595,Kst_srPw7GdYydMFYdCtzw,25,2015-01-10 00:06:25,,2.85


Por usuario

In [20]:
df_y1['user_id'].value_counts()

T5gvmAML_EAA9invlLymJg    5
-J_wra_v8PaPfy2r4_LHiw    5
a25SLDRY8NLhISUucSBZJg    5
zWeCH5pffPS1adw57xzaWA    5
vRYjn0Ip0Zu7SnJYQp9P3Q    5
                         ..
0SGduwsWeTWeL9Sj6pGfyw    1
ManF7sVLflO-i9qWh-oD5g    1
8WCxUvUu8ECp4779Sj5AkA    1
wmSh-O8JDehKBc4QsVUFuw    1
wL5jPrLRVCK_Pmo4lM1zpA    1
Name: user_id, Length: 1987897, dtype: int64

Por determinado usuario

In [21]:
filtro=df_y1['user_id']=='-J_wra_v8PaPfy2r4_LHiw'
df_y1[filtro]

,user_id,review_count,yelping_since,elite,average_stars
27028,-J_wra_v8PaPfy2r4_LHiw,114,2011-06-02 23:09:27,"2015,2016,2017,2018",3.69
2017125,-J_wra_v8PaPfy2r4_LHiw,114,2011-06-02 23:09:27,"2015,2016,2017,2018",3.69
2038225,-J_wra_v8PaPfy2r4_LHiw,114,2011-06-02 23:09:27,"2015,2016,2017,2018",3.69
2059325,-J_wra_v8PaPfy2r4_LHiw,114,2011-06-02 23:09:27,"2015,2016,2017,2018",3.69
2080425,-J_wra_v8PaPfy2r4_LHiw,114,2011-06-02 23:09:27,"2015,2016,2017,2018",3.69


In [22]:
value_counts = df_y1['user_id'].value_counts()
value_counts[value_counts > 1].sum()

161000

In [23]:
value_counts = df_y1['user_id'].value_counts()
values_repeated_more_than_two = value_counts[value_counts > 1]
df_y1_repeated_more_than_two = df_y1[df_y1['user_id'].isin(values_repeated_more_than_two.index)].index
df_y1_user_unicos=df_y1.drop(df_y1_repeated_more_than_two)

In [24]:
df_y1_user_unicos['user_id'].value_counts()

LhfpZHG5fcsMtKzkR7cygQ    1
9xvCtmJ9g3DbueDEPo2aUA    1
Gho_Vyils5T1Ly4df89biA    1
-uvWZEqmkzTkvB369iue7g    1
JcKGEypmas9r1dIQD3Icxg    1
                         ..
0zQHKAOSDzejJz8g_f9icw    1
0EOUDk8D5UOZevLyQuBQEA    1
0SGduwsWeTWeL9Sj6pGfyw    1
ManF7sVLflO-i9qWh-oD5g    1
wL5jPrLRVCK_Pmo4lM1zpA    1
Name: user_id, Length: 1944597, dtype: int64

In [25]:
value_counts = df_y1['user_id'].value_counts()
value_counts[value_counts > 1].sum()

161000

In [26]:
df_y1.duplicated().sum()

117700

Elimino las filas duplicadas

In [27]:
df_y1.drop_duplicates(inplace=True)

In [28]:
df_y1.duplicated().sum()

0

In [29]:
value_counts = df_y1['user_id'].value_counts()
value_counts[value_counts > 1].sum()

0

**FILAS Y VALORES RESULTANTES (modificable)**

In [30]:
df_y1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1987897 entries, 0 to 1987896
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   user_id        object 
 1   review_count   int64  
 2   yelping_since  object 
 3   elite          object 
 4   average_stars  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 91.0+ MB


## **Carga de los datos resultantes**

In [31]:
df_y1.to_parquet('user_y.parquet', index=False, compression='gzip')